In [1]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import sqlite3

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)

app = Bottle()

@app.route('/static/:path#.+#', name='static')
def static(path):
    return static_file(path, root='./static')

@app.route('/')
@app.route('/index/')
def index():
    return template('index')
    
@app.route('/')
@app.route('/login/')
@app.route('/login/')
@app.route('/login', method='post')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):  
        redirect('/main')
        return True
    else:
        return template('login')
    return template('login')

@app.route('/register1')
def register1():
    return template('register1')

def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

app.run(host='localhost', port=63700, reloader=False, debug=False)

Bottle v0.12.23 server starting up (using WSGIRefServer())...
Listening on http://localhost:63700/
Hit Ctrl-C to quit.

Traceback (most recent call last):
  File "c:\Users\kacpe\AppData\Local\Programs\Python\Python311\Lib\site-packages\bottle.py", line 876, in _handle
    return route.call(**args)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\kacpe\AppData\Local\Programs\Python\Python311\Lib\site-packages\bottle.py", line 1756, in wrapper
    rv = callback(*a, **ka)
         ^^^^^^^^^^^^^^^^^^
  File "C:\Users\kacpe\AppData\Local\Temp\ipykernel_15588\386681002.py", line 39, in login
    if (CheckIfCorrectCredentials(loginName,password)==True):
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kacpe\AppData\Local\Temp\ipykernel_15588\386681002.py", line 57, in CheckIfCorrectCredentials
    c.execute("Select * from users where loginName==? and password==?",params)
sqlite3.OperationalError: no such table: users
127.0.0.1 - - [02/Jan/2023 23:13:11] "GET / HTTP/1.1" 500 

Function to login, from example 06_WebappExtra

In [2]:
def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

In [ ]:
@app.route('/')
@app.route('/login/')
@app.route('/login', method='POST')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):  
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')